<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/OSI-SAF_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1b_OSI_SAF_sea_ice_C3S_data_access.ipynb" target="_blank"><< Accessing OSI SAF sea ice products from C3S Data store</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_2_OSI_SAF_sea_ice_file_structure.ipynb" target="_blank">Exploring OSI SAF sea ice file structure >></a>

<font color="#138D75">**EUMETSAT OSI SAF Training Service**</font> <br>
**Copyright:** 2024 EUMETSAT <br>
**License:** MIT

<html>
  <div style="width:100%">
    <div style="float:left"><a href="https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.eumetsat.int%2Feumetlab%2Foceans%2Focean-training%2Fsensors%2Flearn-osi-saf-sea-ice/HEAD?urlpath=%2Ftree%2F1_OSI_SAF_sea_ice_introductory%2F1_1_OSI_SAF_sea_ice_FTP_data_access.ipynb"><img src="https://mybinder.org/badge_logo.svg" alt="Open in Binder"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
  </div>
</html>

<div class="alert alert-block alert-success">
<h3>Learn OSI SAF sea ice parameters : Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **<a href="https://data.marine.copernicus.eu/register" target="_blank">A COPERNICUS MARINE SERVICE account</a>** if you are using or plan to use the Cpernicus Marine data.

There are no prerequisite notebooks for this module.
</div>
<hr>

# 1.1b Accessing all OSI SAF sea ice products from Copernicus Marine Data service
### Data used 

| Dataset | EUMETSAT collection ID | OSI SAF website description | OSI SAF identifier |
|:-----------------:|:-----------------:|:-----------------:|:-----------------:|
| Global Sea Ice Concentration (SSMIS) | EO:EUM:DAT:DMSP:OSI-401-B | <a href="https://osi-saf.eumetsat.int/products/osi-401-b" target="_blank">Description</a>   | OSI-401-B |
| Global Sea Ice Edge | EO:EUM:DAT:MULT:OSIEDGBN | <a href="https://osi-saf.eumetsat.int/products/osi-402-d" target="_blank">Description</a> | OSI-402-d | 
| Global Sea Ice Type | EO:EUM:DAT:MULT:OSITYGBN | <a href="https://osi-saf.eumetsat.int/products/osi-403-d" target="_blank">Description</a> | OSI-403-d | 
| Global Sea Ice Emissivity | EO:EUM:DAT:DMSP:SISE50 | <a href="https://osi-saf.eumetsat.int/products/osi-404-a" target="_blank">Description</a> | OSI-404-a | 
| Global Low Resolution Sea Ice Drift | EO:EUM:DAT:MULT:GBL-LR-SIDR | <a href="https://osi-saf.eumetsat.int/products/osi-405-c" target="_blank">Description</a> | OSI-405-c | 
| Medium Resolution Sea Ice Drift | EO:EUM:DAT:METOP:NH-MR-SID | <a href="https://osi-saf.eumetsat.int/products/osi-407-a" target="_blank">Description</a> | OSI-407-a | 
| Global Sea Ice Concentration (AMSR-2) | EO:EUM:DAT:GCOM:OSI-408 | <a href="https://osi-saf.eumetsat.int/products/osi-408-a" target="_blank">Description</a> | OSI-408-a | 
| Level 2 PMW sea ice concentration | - | <a href="https://osi-saf.eumetsat.int/products/osi-410-a" target="_blank">Description</a> | OSI-410-a | 
| Global Sea Ice Concentration interim climate data record, release 3 | EO:EUM:DAT:0645 | <a href="https://osi-saf.eumetsat.int/products/osi-430-a" target="_blank">Description</a> | OSI-430-a 
| Global Sea Ice Concentration climate data record (SMMR/SSMI/SSMIS), release 3 | EO:EUM:DAT:0826 | <a href="https://osi-saf.eumetsat.int/products/osi-450-a" target="_blank">Description</a> | OSI-450-a 
| Global Sea Ice Concentration climate data record (AMSR), release 3 | EO:EUM:DAT:0797 | <a href="https://osi-saf.eumetsat.int/products/osi-458" target="_blank">Description</a> | OSI-458 
| Sea ice index | - | <a href="https://osi-saf.eumetsat.int/products/osi-420" target="_blank">Description</a> | OSI-420 

### Learning outcomes

At the end of this notebook you will know;
* How to download different OSI SAF Sea Ice Products using FTP access

### Outline

Data from OSI SAF is available through multiple sources. Here we will guide you through FTP access to OSI SAF Sea Ice Data.

<div class="alert alert-info" role="alert">

## <a id='TOC-TOP'></a>Contents

</div>
    
 1. [Creating our workspace](#section1)
 1. [Setting login information](#section2)
 1. [Downloading via FTP requests](#section3)

<hr>

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Creating the workspace
[Back to top](#TOC-TOP)

</div>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [1]:
# library imports 
import os           # a library that allows to access to basic operating system commands like making directories
import json         # a library that helps with JSON format files
import ftplib       # a library that allows to handle ftp requests

Next we will create a download directory to store the products we will download in this notebook.

In [2]:
import lxml.html
import requests as rq

def copernicus_marine_login(url, user, pwd):
    """
    Open a connection to Copernicus Marine CAS with user credentials.

    Parameters
    ----------
    username : str
        Copernicus Marine Username, provided for free from https://marine.copernicus.eu .
    password : str
        Copernicus Marine Password, provided for free from https://marine.copernicus.eu .

    Returns
    -------
    bool
        Returns ``conn_session`` if credentials are correct, ``False`` otherwise.

    """
    params = {'service': url}
    cmems_cas_url = 'https://cmems-cas.cls.fr/cas/login'
    conn_session = rq.session()
    login_session = conn_session.get(cmems_cas_url, params=params)
    login_from_html = lxml.html.fromstring(login_session.text)
    hidden_elements_from_html = login_from_html.xpath(
        '//form//input[@type="hidden"]')
    payload = {
        he.attrib['name']: he.attrib['value']
        for he in hidden_elements_from_html
        }
    payload['username'] = user
    payload['password'] = pwd
    conn_session.post(cmems_cas_url, data=payload, params=params)
    return conn_session

def get_data(url, output_filename, conn_session, csv=False):
    if 'productdownload' in url:
        url = f'{url}&mode=console'
        if csv:
            postprocess = True
    else:
        postprocess = False
    req = conn_session.get(url, stream=True)
    with open(output_filename,'wb') as of:
        for chunk in req.iter_content(chunk_size=1000000):
            of.write(chunk)
    if postprocess:
        try:
            from cmemsapi import cmemsapi as cmust
        except:
            print('Please install cmust if you want to convert to .csv by issuing in your terminal:\npip install cmemsapi')
            raise SystemExit
        cmust.to_csv(output_filename, output_filename)
    return True

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Setting Username and Password
[Back to top](#TOC-TOP)

</div>


We will access OSI SAF data from the OSI SAF High Latitude Centre FTP server.

In order to allow us to download data, we need to provide our credentials. We can do this in two ways; either by creating a file called `.eumetsat_osi_saf_sea_ice_credentials` in our home directory (*option 1 - recommended*) or by supplying our credentials directly in this script (*option 2*). 

#### Option 1: creating  `.eumetsat_osi_saf_sea_ice_credentials` in our home directory

For most computer systems the home directory can be found at the path \user\username, /users/username, or /home/username depending on your operating system.

In this file we need to add the following information exactly as follows;

```
{
"username": "<provided_username>",
"password": "<provided_password>"
}
```

You must replace `<provided_username>` and `<provided_password>` with the information provided to you after you have registered to the OSI SAF website and mentionned your interest in Sea Ice data. 

Once you have created an <a href="https://osi-saf.eumetsat.int/register" target="_blank">EUMETSAT OSI SAF account</a>, the generic password and credentials to access OSI SAF High Latitude server should have been sent to you via email.

Make sure to save the file without any kind of extension.

Once you have done this, you can read in your credentials using the commands in the following cell. These will be used to generate a time-limited token, which will refresh itself when it expires.

Reading credential information

In [3]:
# read credentials
with open(os.path.join(os.path.expanduser("~"),'.eumetsat_osi_saf_sea_ice_credentials')) as json_file:
    credentials = json.load(json_file)

Loading credentials

In [4]:
username = credentials['username'] 
password = credentials['password']

#### Option 2: provide credentials directly

You can provide your credentials directly as follows; by uncommenting the following lines. 

In [5]:
#username = "<provided_username>"
#password = "<provided_password>"

You must replace `<provided_username>` and `<provided_password>` with the information provided to you after you have registered to the OSI SAF website and mentionned your interest in Sea Ice data. 

Once you have created an <a href="https://osi-saf.eumetsat.int/register" target="_blank">EUMETSAT OSI SAF account</a>, the generic password and credentials to access OSI SAF High Latitude server should have been sent to you via email.

*Note: this method is convenient in the short term, but is not really recommended as you have to put your provided username and password in this notebook, and run the risk of accidentally sharing them. This method also requires you to authenticate on a notebook-by-notebook basis.*

<div class="alert alert-info" role="alert">

## <a id='section3'></a>3. Launch an FTP request
[Back to top](#TOC-TOP)

</div>

### Global Sea Ice Concentration (SSMIS) :

Let's start by creating a specific folder to hold the Sea Ice Concentration products

In [6]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_SSMIS")
os.makedirs(download_dir, exist_ok=True)

Sea Ice Concentration products exist for each hemisphere.

Let's look at the Northern hemisphere Arctic Sea Ice Concentration

In [7]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_SSMIS/nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [8]:
path = '/prod/ice/conc/'
file_pattern = 'ice_conc_nh_polstere-100_multi_'

In [9]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_conc_SSMIS', 'nh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

Let's look at the Southern hemisphere Antarctic Sea Ice Concentration

In [10]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_SSMIS/sh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [11]:
path = '/prod/ice/conc/'
file_pattern = 'ice_conc_sh_polstere-100_multi_'

In [12]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_conc_SSMIS', 'sh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Global Sea Ice Type :

Let's start by creating a specific folder to hold the Sea Ice Type products

In [13]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_type")
os.makedirs(download_dir, exist_ok=True)

Sea Ice Type products exist for each hemisphere.

Let's look at the Northern hemisphere Arctic Sea Ice Type

In [14]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_type", "nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [15]:
path = '/prod/ice/type/'
file_pattern = 'ice_type_nh_polstere-100_multi_'

In [16]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_type', 'nh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

Let's look at the Southern hemisphere Antarctic Sea Ice Type

In [17]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_type", "sh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [18]:
path = '/prod/ice/type/'
file_pattern = 'ice_type_sh_polstere-100_multi_'

In [19]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_type', 'sh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Global Sea Ice Edge :

Let's start by creating a specific folder to hold the Sea Ice Edge products

In [20]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_edge")
os.makedirs(download_dir, exist_ok=True)

Sea Ice Edge products exist for each hemisphere.

Let's look at the Northern hemisphere Arctic Sea Ice Edge

In [21]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_edge", "nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [22]:
path = '/prod/ice/edge/'
file_pattern = 'ice_edge_nh_polstere-100_multi_'

In [23]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_edge', 'nh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

Let's look at the Southern hemisphere Antarctic Sea Ice Edge

In [24]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_edge", "sh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [25]:
path = '/prod/ice/edge/'
file_pattern = 'ice_edge_sh_polstere-100_multi_'

In [26]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_edge', 'sh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Global Sea Ice Emissivity :

Let's start by creating a specific folder to hold the Sea Ice Emissivity products

In [27]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_emissivity")
os.makedirs(download_dir, exist_ok=True)

Sea Ice Emmissivity products exist for each hemisphere.

Let's look at the Northern hemisphere Arctic Sea Ice Emmissivity

In [28]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_emissivity", "nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [29]:
path = '/prod/ice/emis/'
file_pattern = 'ice_emis_nh_stere-100_ssmis_'

In [30]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_emissivity', 'nh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

Let's look at the Southern hemisphere Antarctic Sea Ice Emissivity

In [31]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_emissivity", "sh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [32]:
path = '/prod/ice/emis/'
file_pattern = 'ice_emis_sh_stere-100_ssmis_'

In [33]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_emissivity', 'sh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Global Low Resolution Sea Ice Drift :

Let's start by creating a specific folder to hold the Sea Ice Drift low resolution products

In [34]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_drift_lr")
os.makedirs(download_dir, exist_ok=True)

Sea Ice Drift products exist for each hemisphere.

Let's look at the Northern hemisphere Arctic Sea Ice Drift

In [35]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_drift_lr", "nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [36]:
path = '/prod/ice/drift_lr/merged'
file_pattern = 'ice_drift_nh_polstere-625_'

In [37]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_drift_lr', 'nh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Medium Resolution Sea Ice Drift
Let's start by creating a specific folder to hold the Sea Ice Drift Medium resolution products

In [38]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_drift_mr")
os.makedirs(download_dir, exist_ok=True)

Let's look at the Northern hemisphere Arctic Sea Ice Drift

In [39]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_drift_mr", "nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [40]:
path = '/prod/ice/drift_mr'
file_pattern = 'ice_drift_nh_polstere-200_'

In [41]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_drift_mr', 'nh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Global Sea Ice Concentration (AMSR-2)
Let's start by creating a specific folder to hold the Global Sea Ice Concentration (AMSR-2) products

In [42]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_AMSR")
os.makedirs(download_dir, exist_ok=True)

Global Sea Ice Concentration (AMSR-2) products exist for each hemisphere.

Let's look at the Northern hemisphere

In [43]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_AMSR", "nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [44]:
path = '/archive/ice/conc_amsr/2023/06'
file_pattern = 'ice_conc_nh_polstere-100_amsr2_'

In [45]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_conc_AMSR', 'nh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

Let's look at the Southern hemisphere 

In [46]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_AMSR", "sh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [47]:
path = '/archive/ice/conc_amsr/2023/06'
file_pattern = 'ice_conc_sh_polstere-100_amsr2_'

In [48]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_conc_AMSR', 'sh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Level 2 PMW sea ice concentration
Let's start by creating a specific folder to hold theLevel 2 PMW sea ice concentration products

In [49]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_PMW")
os.makedirs(download_dir, exist_ok=True)

Level 2 PMW sea ice concentration product exist for SSMIS and AMSR-2 instruments

Let's look at the AMSR intrument

In [50]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_PMW", "AMSR")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [51]:
path = '/archive/ice/conc_l2_amsr/2023/06/16'
file_pattern = 'ice_conc_l2_amsr2_gw1_'

In [52]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
        ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_conc_PMW', 'AMSR', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

Let's look at the SSMIS intrument

In [53]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_PMW", "SSMIS")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [54]:
path = '/archive/ice/conc_l2_ssmis/2023/06/16'
file_pattern = 'ice_conc_l2_ssmis_'

In [55]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
        ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_conc_PMW', 'SSMIS', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Global Sea Ice Concentration interim climate data record, release 3
Let's start by creating a specific folder to hold the Global Sea Ice Concentration interim climate data record, release 3 products

In [56]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_interim")
os.makedirs(download_dir, exist_ok=True)

Global Sea Ice Concentration (AMSR-2) products exist for each hemisphere.

Let's look at the Northern hemisphere

In [57]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_interim", "nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [58]:
path = '/reprocessed/ice/conc-cont-reproc/v3p0/2023/06'
file_pattern = 'ice_conc_nh_ease2-250_icdr-v3p0_'

In [59]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_conc_interim', 'nh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

Let's look at the Southern hemisphere

In [60]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_interim", "sh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [61]:
path = '/reprocessed/ice/conc-cont-reproc/v3p0/2023/06'
file_pattern = 'ice_conc_sh_ease2-250_icdr-v3p0_'

In [62]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_conc_interim', 'sh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Global Sea Ice Concentration climate data record (SMMR/SSMI/SSMIS), release 3
Let's start by creating a specific folder to hold the Global Sea Ice Concentration climate data record (SMMR/SSMI/SSMIS), release 3 products


In [63]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_S")
os.makedirs(download_dir, exist_ok=True)

Global Sea Ice Concentration climate data record (SMMR/SSMI/SSMIS), release 3 products exist for each hemisphere.

Let's look at the Northern hemisphere

In [64]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_S", "nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [65]:
path = '/reprocessed/ice/conc/v3p0/2020/12'
file_pattern = 'ice_conc_nh_ease2-250_cdr-v3p0_'

In [66]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_conc_S', 'nh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

Let's look at the Southern hemisphere

In [67]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_S", "sh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [68]:
path = '/reprocessed/ice/conc/v3p0/2020/12'
file_pattern = 'ice_conc_sh_ease2-250_cdr-v3p0_'

In [69]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(),'products', 'sea_ice_conc_S', 'sh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Global Sea Ice Concentration climate data record (AMSR), release 3
Let's start by creating a specific folder to hold the Global Sea Ice Concentration climate data record (AMSR), release 3 products.

Data available until 2020-12-31


In [70]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_AMSR_release")
os.makedirs(download_dir, exist_ok=True)

Global Sea Ice Concentration climate data record (AMSR), release 3 products exist for each hemisphere.

Let's look at the Northern hemisphere

In [71]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_AMSR_release", "nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [72]:
path = '/reprocessed/ice/conc_amsr/v3p0/2020/12'
file_pattern = 'ice_conc_nh_ease2-250_cdr-v3p0-amsr_'

In [73]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_conc_AMSR_release', 'nh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

Let's look at the Southern hemisphere

In [74]:
download_dir = os.path.join(os.getcwd(), "products", "sea_ice_conc_AMSR_release", "sh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [75]:
path = '/reprocessed/ice/conc_amsr/v3p0/2020/12'
file_pattern = 'ice_conc_sh_ease2-250_cdr-v3p0-amsr_'

In [76]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'sea_ice_conc_AMSR_release', 'sh', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

<hr>
<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1b_OSI_SAF_sea_ice_C3S_data_access.ipynb" target="_blank"><< Accessing OSI SAF sea ice products from C3S Data store</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_2_OSI_SAF_sea_ice_file_structure.ipynb" target="_blank">Exploring OSI SAF sea ice file structure >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>